In [1]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env.yaml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})
os.environ['DATABASE_URL'] = 'sqlite:///:memory:'
os.environ['WSL_DISTRIBUTION'] = 'Ubuntu'

from hemlock import push_app_context

app = push_app_context()

In [2]:
[os.remove(t) for t in app.tmpfiles if os.path.exists(t)]

[]

In [7]:
from hemlock import Page, Binary, Blank, Label, Validate as V
from hemlock.tools import show_on_event

convinced = Binary(
    '''
    <p><b>FAQ</b>. Should I bid less for the model than I'm actually willing to pay for it?</p>

    <p><b>Answer: NO!</b> We determine whether to accept your bid using an <i>incentive-compatible mechanism.</i> This is a fancy term economists use which means that you're best off when you bid exactly as much as the thing is worth to you. There's no way to game the system. Attempting to do so will make you worse off.</p>
    ''',
    [
        "I'm convinced. I'm going to bid exactly as much as the model is worth to me.",
        "I'm not convinced. Please explain more."
    ],
    var='Convinced', data_rows=-1, inline=False, validate=V.require()
)
explain = Label(
    '''
    <p>We determine whether to accept your bid using the <i>Becker-DeGroot-Marschak mechanism</i> or <i>Vickrey auction</i>. Developed by a Nobel Prize-winning economist, it's mathematically proven to be 'un-gameable'. It's not important to understand how it works. What <i>is</i> important is to understand that you're best off when you bid exactly as much as the model is worth to you.</p>
    
    <p>For the truly skeptical, <a href="https://en.wikipedia.org/wiki/Vickrey_auction#Proof_of_dominance_of_truthful_bidding" target="_blank">read this</a> mathematical proof of why you should bid exactly as much as the model is worth to you.</p>
    '''
)
show_on_event(explain, convinced, 0)
Page(
    Label(
        '''
        <p>You will now make {} predictions. You will not receive feedback, and these predictions <i>will</i> determine your bonus.</p>

        <p>Additionally, your free trial using the computer model has expired. To continue using the model, you must place a bid for it.</p>

        <p>If we accept your bid, we'll reduce your bonus and allow you to continue using the model. If we reject your bid, we won't reduce your bonus, but you can no longer see the model's predictions. The larger your bid, the more likely we are to accept it.</p>
        '''
    ),
    convinced, explain,
    Blank(
        ('''
        <p>Recall that there is a 1 in 10 chance we will select you to receive a bonus. From previous studies, we estimate that <b>the average participant will earn a $1 larger bonus by continuing to use the model</b>.</p>

        <p>How much are you willing to bid (pay) to continue using the model?</p>

        <ul>
            <li>I am willing to pay up to ''', ''' cents to use the model</li>
            <li>I am unwilling to pay more than ''', ''' cents to use the model</li>
        </ul>
        '''),
        var='WTP', data_rows=-1, blank_empty='_____', append='cents'
    )
).preview()

<Page (transient 140495698213464)>